In [1]:
import pandas as pd
import ast

In [2]:
df=pd.read_csv('attribution_allocation_student_data.csv',encoding='utf-8')
df

,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier
0,True,email,direct,social,NaN,NaN,1
1,True,social,direct,organic_search,NaN,NaN,1
2,True,organic_search,display,social,NaN,NaN,1
3,True,social,direct,NaN,NaN,NaN,1
4,True,social,display,direct,NaN,NaN,1
...,...,...,...,...,...,...,...
218407,False,direct,NaN,NaN,NaN,NaN,3
218408,False,email,direct,organic_search,NaN,NaN,3
218409,False,direct,organic_search,display,NaN,NaN,3
218410,False,social,email,direct,paid_search,organic_search,3


In [3]:
spend_df=pd.read_csv('channel_spend_student_data.csv')
spend_df

,tier,spend by channel
0,1,"{'social': 50, 'organic_search': 0, 'referral'..."
1,2,"{'social': 100, 'organic_search': 0, 'referral..."
2,3,"{'social': 150, 'organic_search': 0, 'referral..."
3,total,"{'social': 300, 'organic_search': 0, 'referral..."


In [4]:
total=spend_df[spend_df['tier']=='total']
t1=spend_df[spend_df['tier']=='1']
t2=spend_df[spend_df['tier']=='2']
t3=spend_df[spend_df['tier']=='3']

In [5]:
total_dict=ast.literal_eval(total['spend by channel'].item())
t1_dict=ast.literal_eval(t1['spend by channel'].item())
t2_dict=ast.literal_eval(t2['spend by channel'].item())
t3_dict=ast.literal_eval(t3['spend by channel'].item())

In [6]:
df_converted=df[df.convert_TF==True]
df_converted

,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier
0,True,email,direct,social,NaN,NaN,1
1,True,social,direct,organic_search,NaN,NaN,1
2,True,organic_search,display,social,NaN,NaN,1
3,True,social,direct,NaN,NaN,NaN,1
4,True,social,display,direct,NaN,NaN,1
...,...,...,...,...,...,...,...
213311,True,display,organic_search,email,direct,NaN,3
213312,True,direct,NaN,NaN,NaN,NaN,3
213313,True,direct,organic_search,email,NaN,NaN,3
213314,True,social,direct,display,organic_search,NaN,3


# Part 1: Attribution

### Method 1: First Interaction

In [7]:
df_converted['first_interaction']=df_converted.apply(lambda row: row.touch_1,axis=1)
df_converted

d:\programs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier,first_interaction
0,True,email,direct,social,NaN,NaN,1,email
1,True,social,direct,organic_search,NaN,NaN,1,social
2,True,organic_search,display,social,NaN,NaN,1,organic_search
3,True,social,direct,NaN,NaN,NaN,1,social
4,True,social,display,direct,NaN,NaN,1,social
...,...,...,...,...,...,...,...,...
213311,True,display,organic_search,email,direct,NaN,3,display
213312,True,direct,NaN,NaN,NaN,NaN,3,direct
213313,True,direct,organic_search,email,NaN,NaN,3,direct
213314,True,social,direct,display,organic_search,NaN,3,social


In [8]:
channel=[]
count=[]
for k,v in df_converted.groupby('first_interaction'):
    channel.append(k)
    count.append(len(v[v.convert_TF==True]))
result_df=pd.DataFrame({'first_interaction':channel,'count_convert':count})
result_df

,first_interaction,count_convert
0,direct,550
1,display,434
2,email,357
3,organic_search,675
4,paid_search,12
5,referral,10
6,social,340


In [9]:
result_df['total_spend']=result_df['first_interaction'].apply(lambda x: total_dict[x])
result_df

,first_interaction,count_convert,total_spend
0,direct,550,0
1,display,434,300
2,email,357,300
3,organic_search,675,0
4,paid_search,12,300
5,referral,10,300
6,social,340,300


In [10]:
result_df['CAC']=result_df['total_spend']/result_df['count_convert']
result_df=result_df.sort_values(by='CAC',ascending=True)
result_df

,first_interaction,count_convert,total_spend,CAC
0,direct,550,0,0.000000
3,organic_search,675,0,0.000000
1,display,434,300,0.691244
2,email,357,300,0.840336
6,social,340,300,0.882353
4,paid_search,12,300,25.000000
5,referral,10,300,30.000000


display<email<social<paid_search<referral

### Method 2: Last Interaction

In [11]:
def last_interaction(row):
    if pd.notna(row.touch_5) == True:
        result = row.touch_5
    elif pd.notna(row.touch_4) == True:
        result = row.touch_4
    elif pd.notna(row.touch_3) == True:
        result = row.touch_3
    elif pd.notna(row.touch_2) == True:
        result = row.touch_2
    else:
        result = row.touch_1
    return result

In [12]:
df_converted['last_interaction']=df_converted.apply(last_interaction,axis=1)
df_converted

d:\programs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier,first_interaction,last_interaction
0,True,email,direct,social,NaN,NaN,1,email,social
1,True,social,direct,organic_search,NaN,NaN,1,social,organic_search
2,True,organic_search,display,social,NaN,NaN,1,organic_search,social
3,True,social,direct,NaN,NaN,NaN,1,social,direct
4,True,social,display,direct,NaN,NaN,1,social,direct
...,...,...,...,...,...,...,...,...,...
213311,True,display,organic_search,email,direct,NaN,3,display,direct
213312,True,direct,NaN,NaN,NaN,NaN,3,direct,direct
213313,True,direct,organic_search,email,NaN,NaN,3,direct,email
213314,True,social,direct,display,organic_search,NaN,3,social,organic_search


In [13]:
channel=[]
count=[]
for k,v in df_converted.groupby('last_interaction'):
    channel.append(k)
    count.append(len(v[v.convert_TF==True]))
result_df=pd.DataFrame({'last_interaction':channel,'count_convert':count})
result_df

,last_interaction,count_convert
0,direct,613
1,display,406
2,email,323
3,organic_search,662
4,paid_search,12
5,referral,10
6,social,352


In [14]:
result_df['total_spend']=result_df['last_interaction'].apply(lambda x: total_dict[x])
result_df

,last_interaction,count_convert,total_spend
0,direct,613,0
1,display,406,300
2,email,323,300
3,organic_search,662,0
4,paid_search,12,300
5,referral,10,300
6,social,352,300


In [15]:
result_df['CAC']=result_df['total_spend']/result_df['count_convert']
result_df=result_df.sort_values(by='CAC',ascending=True)
result_df

,last_interaction,count_convert,total_spend,CAC
0,direct,613,0,0.000000
3,organic_search,662,0,0.000000
1,display,406,300,0.738916
6,social,352,300,0.852273
2,email,323,300,0.928793
4,paid_search,12,300,25.000000
5,referral,10,300,30.000000


display<social<email<paid_search<referral

### Method 3: Last Non-Direct Interaction

In [16]:
def last_nd_interaction(row):
    if (pd.notna(row.touch_5) == True) and (row.touch_5 != 'direct') :
        result = row.touch_5
    elif (pd.notna(row.touch_4) == True) and (row.touch_4 != 'direct'):
        result = row.touch_4
    elif (pd.notna(row.touch_3) == True) and (row.touch_3 != 'direct') :
        result = row.touch_3
    elif (pd.notna(row.touch_2) == True) and (row.touch_2 != 'direct'):
        result = row.touch_2
    else:
        result = row.touch_1
    return result

In [17]:
df_converted['last_nd_interaction']=df_converted.apply(last_nd_interaction,axis=1)
df_converted

d:\programs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier,first_interaction,last_interaction,last_nd_interaction
0,True,email,direct,social,NaN,NaN,1,email,social,social
1,True,social,direct,organic_search,NaN,NaN,1,social,organic_search,organic_search
2,True,organic_search,display,social,NaN,NaN,1,organic_search,social,social
3,True,social,direct,NaN,NaN,NaN,1,social,direct,social
4,True,social,display,direct,NaN,NaN,1,social,direct,display
...,...,...,...,...,...,...,...,...,...,...
213311,True,display,organic_search,email,direct,NaN,3,display,direct,email
213312,True,direct,NaN,NaN,NaN,NaN,3,direct,direct,direct
213313,True,direct,organic_search,email,NaN,NaN,3,direct,email,email
213314,True,social,direct,display,organic_search,NaN,3,social,organic_search,organic_search


In [18]:
channel=[]
count=[]
for k,v in df_converted.groupby('last_nd_interaction'):
    channel.append(k)
    count.append(len(v[v.convert_TF==True]))
result_df=pd.DataFrame({'last_nd_interaction':channel,'count_convert':count})
result_df

,last_nd_interaction,count_convert
0,direct,49
1,display,525
2,email,426
3,organic_search,896
4,paid_search,14
5,referral,14
6,social,454


In [19]:
result_df['total_spend']=result_df['last_nd_interaction'].apply(lambda x: total_dict[x])
result_df

,last_nd_interaction,count_convert,total_spend
0,direct,49,0
1,display,525,300
2,email,426,300
3,organic_search,896,0
4,paid_search,14,300
5,referral,14,300
6,social,454,300


In [20]:
result_df['CAC']=result_df['total_spend']/result_df['count_convert']
result_df=result_df.sort_values(by='CAC',ascending=True)
result_df

,last_nd_interaction,count_convert,total_spend,CAC
0,direct,49,0,0.000000
3,organic_search,896,0,0.000000
1,display,525,300,0.571429
6,social,454,300,0.660793
2,email,426,300,0.704225
4,paid_search,14,300,21.428571
5,referral,14,300,21.428571


display<social<email<paid_search=referral

# Part 2: Allocation

### choose first interaction method

In [21]:
tier_dist = df_converted.groupby(['first_interaction','tier']).agg({'convert_TF': 'count'})

tier_dist = tier_dist.reset_index()

tier_dist

,first_interaction,tier,convert_TF
0,direct,1,94
1,direct,2,171
2,direct,3,285
3,display,1,92
4,display,2,167
5,display,3,175
6,email,1,67
7,email,2,116
8,email,3,174
9,organic_search,1,113


In [22]:
def tier_spend(row):
    if row.tier==1:
        spend_amt=t1_dict[row.first_interaction]
    elif row.tier==2:
        spend_amt=t2_dict[row.first_interaction]
    elif row.tier==3:
        spend_amt=t3_dict[row.first_interaction]
    return spend_amt

In [23]:
tier_dist['spend']=tier_dist.apply(tier_spend,axis=1)
tier_dist

,first_interaction,tier,convert_TF,spend
0,direct,1,94,0
1,direct,2,171,0
2,direct,3,285,0
3,display,1,92,50
4,display,2,167,100
5,display,3,175,150
6,email,1,67,50
7,email,2,116,100
8,email,3,174,150
9,organic_search,1,113,0


In [24]:
tier_data=tier_dist[(tier_dist.first_interaction != 'direct')&(tier_dist.first_interaction != 'organic_search')]
tier_data['marginal_spend']=50
tier_data=tier_data.reset_index(drop=True)
tier_data

d:\programs\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,first_interaction,tier,convert_TF,spend,marginal_spend
0,display,1,92,50,50
1,display,2,167,100,50
2,display,3,175,150,50
3,email,1,67,50,50
4,email,2,116,100,50
5,email,3,174,150,50
6,paid_search,1,2,50,50
7,paid_search,2,5,100,50
8,paid_search,3,5,150,50
9,referral,1,2,50,50


In [25]:
result_df=pd.DataFrame()
for k,v in tier_data.groupby('first_interaction'):
    v['convert_TF_1']=v['convert_TF'].shift(1)
    v['convert_TF_1']=v['convert_TF_1'].fillna(0)
    v['marginal_convert']=v['convert_TF']-v['convert_TF_1']
    result_df=pd.concat([result_df,v],axis=0)

d:\programs\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\programs\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
d:\programs\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [26]:
result_df=result_df.drop('convert_TF_1',axis=1)
result_df['marginal_CAC']=result_df['marginal_spend']/result_df['marginal_convert']
result_df

,first_interaction,tier,convert_TF,spend,marginal_spend,marginal_convert,marginal_CAC
0,display,1,92,50,50,92.0,0.543478
1,display,2,167,100,50,75.0,0.666667
2,display,3,175,150,50,8.0,6.250000
3,email,1,67,50,50,67.0,0.746269
4,email,2,116,100,50,49.0,1.020408
5,email,3,174,150,50,58.0,0.862069
6,paid_search,1,2,50,50,2.0,25.000000
7,paid_search,2,5,100,50,3.0,16.666667
8,paid_search,3,5,150,50,0.0,inf
9,referral,1,2,50,50,2.0,25.000000


In [27]:
result_df.sort_values(by='marginal_CAC',ascending=True)

,first_interaction,tier,convert_TF,spend,marginal_spend,marginal_convert,marginal_CAC
10,referral,2,1,100,50,-1.0,-50.000000
0,display,1,92,50,50,92.0,0.543478
1,display,2,167,100,50,75.0,0.666667
12,social,1,71,50,50,71.0,0.704225
3,email,1,67,50,50,67.0,0.746269
5,email,3,174,150,50,58.0,0.862069
4,email,2,116,100,50,49.0,1.020408
13,social,2,118,100,50,47.0,1.063830
14,social,3,151,150,50,33.0,1.515152
2,display,3,175,150,50,8.0,6.250000


In [30]:
result_df.sort_values(by='marginal_CAC',ascending=True).pivot_table(index = 'tier', columns = 'first_interaction', values ='marginal_CAC', aggfunc ='sum')

first_interaction,display,email,paid_search,referral,social
tier,,,,,
1,0.543478,0.746269,25.000000,25.000000,0.704225
2,0.666667,1.020408,16.666667,-50.000000,1.063830
3,6.250000,0.862069,inf,8.333333,1.515152
